# Transform the detection results into the style Yiming could use to draw figure

Store the results in scene_frame.npy

Target Data Style:
{"mean": N\*4\*2, "covar": N\*4\*2\*2, "groundtruth": N\*4\*2}

In [1]:
import numpy as np
import torch
import os

In [2]:
def del_detection(det_results):
    det_mean = []
    det_covar = []
    for i in range(len(det_results)):
        for j in range(len(det_results[i])):
            mean = det_results[i][j][:,:8]
            covar = det_results[i][j][:,9:]
            mean = mean.reshape((mean.shape[0],4,2))
            covar = covar.reshape((covar.shape[0],4,3))
            u_matrix = np.zeros((covar.shape[0], 4, 2, 2))
            u_matrix[:, :, 0, 0] = np.exp(covar[:, :, 0])
            u_matrix[:, :, 0, 1] = covar[:, :, 1]
            u_matrix[:, :, 1, 1] = np.exp(covar[:, :, 2])
            sigma_inverse = np.matmul(np.transpose(u_matrix, (0,1,3,2)), u_matrix)
            covar_matrix = np.linalg.inv(sigma_inverse)
            covar_matrix = covar_e + (0.5 * covar_a + 0.5 * covar_matrix) * 100.0
            #covar_matrix = covar_matrix * 100.0
            det_mean.append(mean)
            det_covar.append(covar_matrix)
    assert len(det_mean)==1
    return det_mean[0], det_covar[0]

def check_folder(folder_path):
    if not os.path.exists(folder_path):
        os.mkdir(folder_path)
    return folder_path

In [11]:
check_path = "./check/test_loss_mbb_two_step_corner_pair_ind"
com = "lowerbound"
nepoch = 13
data_path = os.path.join(os.path.join(check_path,com + '/no_rsu/'),str(nepoch))
save_path = os.path.join(os.path.join(check_path,com),com)
check_folder(save_path)

'./check/test_loss_mbb_two_step_corner_pair_ind/lowerbound/lowerbound'

In [12]:
mbb_covar_file = "./check/check_loss_mbb_two_step_corner_pair_ind/" +com + "/no_rsu/mbb_covar_05.npy"
covar_data = np.load(mbb_covar_file, allow_pickle=True)
covar_e = covar_data.item()['covar_e']
covar_a = covar_data.item()['covar_a']
print(covar_e)
print(covar_a)

[[0.05040987 0.0010166 ]
 [0.0010166  0.06151561]]
[[1.04491623e-03 3.78390509e-06]
 [3.78390509e-06 9.59303807e-04]]


In [13]:
file_list = os.listdir(data_path + "/result1")
agent_list = ["result{}".format(i) for i in range(1,6)]
for agent in agent_list:
    save_file_folder = save_path + "/" + agent
    check_folder(save_file_folder)
    for scene_frame in file_list:
        if not scene_frame[0].isdigit():
            continue
        one_file = data_path + "/" + agent + "/" + scene_frame
        data = np.load(one_file, allow_pickle=True)
        print("Load data from {}".format(one_file))
        if len(data.item()['det_results_frame']) != 0:
            det_results = data.item()['det_results_frame']
            annotations = data.item()['annotations_frame'][0]['bboxes']
            annotations = annotations.reshape((annotations.shape[0], 4, 2))
            det_mean, det_covar = del_detection(det_results)
            results = {"mean":det_mean, "covar": det_covar, "groundtruth": annotations}
        else:
            results = {"mean":[], "covar": [], "groundtruth": []}
        one_file_save = save_file_folder + "/" + scene_frame
        np.save(one_file_save, results)
        print("save results to {}".format(one_file_save))

Load data from ./check/test_loss_mbb_two_step_corner_pair_ind/lowerbound/no_rsu/13/result1/96_75.npy
save results to ./check/test_loss_mbb_two_step_corner_pair_ind/lowerbound/lowerbound/result1/96_75.npy
Load data from ./check/test_loss_mbb_two_step_corner_pair_ind/lowerbound/no_rsu/13/result1/5_98.npy
save results to ./check/test_loss_mbb_two_step_corner_pair_ind/lowerbound/lowerbound/result1/5_98.npy
Load data from ./check/test_loss_mbb_two_step_corner_pair_ind/lowerbound/no_rsu/13/result1/19_29.npy
save results to ./check/test_loss_mbb_two_step_corner_pair_ind/lowerbound/lowerbound/result1/19_29.npy
Load data from ./check/test_loss_mbb_two_step_corner_pair_ind/lowerbound/no_rsu/13/result1/19_64.npy
save results to ./check/test_loss_mbb_two_step_corner_pair_ind/lowerbound/lowerbound/result1/19_64.npy
Load data from ./check/test_loss_mbb_two_step_corner_pair_ind/lowerbound/no_rsu/13/result1/92_39.npy
save results to ./check/test_loss_mbb_two_step_corner_pair_ind/lowerbound/lowerbound/

In [89]:
scene_frame = "92_39.npy"
agent = "result4"
one_file = data_path + "/" + agent + "/" + scene_frame
data = np.load(one_file, allow_pickle=True)
det_results = data.item()['det_results_frame']
annotations = data.item()['annotations_frame'][0]['bboxes']

IndexError: list index out of range

In [94]:
data.item()['det_results_frame']

[]

In [76]:
annotations = annotations.reshape((annotations.shape[0], 4, 2))
det_mean, det_covar = del_detection(det_results)
results = {"mean":det_mean, "covar": det_covar, "groundtruth": annotations}

In [77]:
save_file_folder = save_path + "/" + agent
one_file_save = save_file_folder + "/" + scene_frame
check_folder(save_file_folder)
np.save(one_file_save, results)
print("save results to {}".format(one_file_save))

save results to ./check/test_loss_mbb_two_step_corner_pair_ind/upperbound/upperbound/result1/19_0.npy
